In [60]:
# Function to locally avoid obstacles, memory implemented
%%run_python

speed0 = 200      # nominal speed
speedGain = 2     # gain used with ground gradient
obstSpeedGain = [6, 4, -2, -4, -10]    # /100
                  # gains used with front proximity sensors 0..4

# Obstacle memory parameters
obstacle_memory = [0, 0, 0, 0, 0]  # memory timer for each sensor
MEMORY_TIME = 200      # cycles to keep avoiding after detection (N * 10ms)
DETECTION_THRESHOLD = 1500  # sensor value to trigger memory

timer_period[0] = 10  # 10ms sampling time

toggle = False

@onevent
def button_center():
    global speed0, obstacle_memory, toggle, speedGain, obstSpeedGain

    if button_center == 1:
        toggle = 1 if toggle == 0 else 0

@onevent
def timer0():
    global prox_ground_delta, prox_horizontal, speed0, speedGain, obstSpeedGain, motor_left_target, motor_right_target, obstacle_memory, MEMORY_TIME, DETECTION_THRESHOLD, toggle

    # acquisition from ground sensor
    diffDelta = prox_ground_delta[1] - prox_ground_delta[0]

    if toggle:
        spLeft = speed0 - speedGain * diffDelta
        spRight = speed0 + speedGain * diffDelta

        # update obstacle memory and apply avoidance
        for i in range(5):
            # Update memory: refresh if obstacle detected, otherwise decay
            if prox_horizontal[i] > DETECTION_THRESHOLD:
                obstacle_memory[i] = MEMORY_TIME
            elif obstacle_memory[i] > 0:
                obstacle_memory[i] -= 1

            # Apply current sensor reading
            spLeft += prox_horizontal[i] * obstSpeedGain[i] // 100
            spRight += prox_horizontal[i] * obstSpeedGain[4 - i] // 100

            # Apply memory-based avoidance (using memory value as if it were a sensor reading)
            if obstacle_memory[i] > 0:
                spLeft += obstacle_memory[i] * obstSpeedGain[i] // 100
                spRight += obstacle_memory[i] * obstSpeedGain[4 - i] // 100

    else:
        spLeft = 0
        spRight = 0

    # motor control
    motor_left_target = spLeft
    motor_right_target = spRight
